In [1]:
import numpy as np
from subprocess import PIPE, run
import matplotlib.pyplot as plt
import os
import textwrap

class ExptBuilder():
    def __init__(self):
        self.__code_path__ = os.environ.get('code')
        self.__temp_exp_path__ = os.path.join(self.__code_path__, "k-exp", "kexp", "experiments", "ml_expt.py")

    def run_expt(self):
        expt_path = self.__temp_exp_path__
        run_expt_command = r"%kpy% & artiq_run " + expt_path
        result = run(run_expt_command, stdout=PIPE, stderr=PIPE, universal_newlines=True, shell=True)
        print(result.returncode, result.stdout, result.stderr)
        os.remove(self.__temp_exp_path__)
        return result.returncode
    
    def write_experiment_to_file(self, program):
        with open(self.__temp_exp_path__, 'w') as file:
            file.write(program)
    
    def fringe_scan_expt(self, f_list):
        script = textwrap.dedent(f"""
        from artiq.experiment import *
        from artiq.experiment import delay
        from kexp import Base
        import numpy as np
        from kexp.util.artiq.async_print import aprint

        from kexp.calibrations.tweezer import tweezer_vpd1_to_vpd2
        from kexp.calibrations.imaging import high_field_imaging_detuning

        class tweezer_load(EnvExperiment, Base):

            def prepare(self):
                Base.__init__(self,setup_camera=True,camera_select='andor',save_data=True)

                self.xvar('t_tof',np.linspace(1200.,3000.,5)*1.e-6)

                self.p.frequency_tweezer_list = {f_list}

                a_list = [.5275,.155]
                self.p.amp_tweezer_list = a_list
                self.p.amp_tweezer_auto_compute = False

                # self.p.t_tof = 2600.e-6
                self.p.N_repeats = 100

                self.p.t_mot_load = .75
                self.p.v_pd_tweezer_1064_rampdown3_end = 1.

                self.camera_params.amp_imaging = .12
                self.camera_params.exposure_time = 10.e-6
                self.p.t_imaging_pulse = self.camera_params.exposure_time

                self.finish_prepare(shuffle=True)

            @kernel
            def scan_kernel(self):

                self.set_high_field_imaging(i_outer=self.p.i_evap3_current)
                # self.dds.imaging.set_dds(amplitude=self.p.amp_imaging)
                # self.set_imaging_detuning(self.p.frequency_detuned_imaging)

                self.switch_d2_2d(1)
                self.mot(self.p.t_mot_load)
                self.dds.push.off()
                self.cmot_d1(self.p.t_d1cmot * s)
                
                self.gm(self.p.t_gm * s)
                self.gm_ramp(self.p.t_gmramp)

                self.magtrap_and_load_lightsheet()

                # feshbach field on, ramp up to field 1  
                self.outer_coil.on()
                delay(1.e-3)
                self.outer_coil.set_voltage()
                self.outer_coil.ramp(t=self.p.t_feshbach_field_rampup,
                                    i_start=0.,
                                    i_end=self.p.i_evap1_current)
                
                # lightsheet evap 1
                self.lightsheet.ramp(t=self.p.t_lightsheet_rampdown,
                                    v_start=self.p.v_pd_lightsheet_rampup_end,
                                    v_end=self.p.v_pd_lightsheet_rampdown_end)
                
                # feshbach field ramp to field 2
                self.outer_coil.ramp(t=self.p.t_feshbach_field_ramp,
                                    i_start=self.p.i_evap1_current,
                                    i_end=self.p.i_evap2_current)
                
                self.ttl.pd_scope_trig.pulse(1.e-6)
                self.tweezer.on(paint=False)
                self.tweezer.ramp(t=self.p.t_tweezer_1064_ramp,
                                v_start=0.,
                                v_end=self.p.v_pd_tweezer_1064_ramp_end,
                                paint=True,keep_trap_frequency_constant=False)
                
                # # lightsheet ramp down (to off)
                self.lightsheet.ramp(t=self.p.t_lightsheet_rampdown2,
                                    v_start=self.p.v_pd_lightsheet_rampdown_end,
                                    v_end=self.p.v_pd_lightsheet_rampdown2_end)
                
                # tweezer evap 1 with constant trap frequency
                self.tweezer.ramp(t=self.p.t_tweezer_1064_rampdown,
                                v_start=self.p.v_pd_tweezer_1064_ramp_end,
                                v_end=self.p.v_pd_tweezer_1064_rampdown_end,
                                paint=True,keep_trap_frequency_constant=True)

                # feshbach field ramp to field 3
                self.outer_coil.ramp(t=self.p.t_feshbach_field_ramp2,
                                    i_start=self.p.i_evap2_current,
                                    i_end=self.p.i_evap3_current)
                
                # tweezer evap 2 with constant trap frequency
                self.tweezer.ramp(t=self.p.t_tweezer_1064_rampdown2,
                                v_start=self.p.v_pd_tweezer_1064_rampdown_end,
                                v_end=self.p.v_pd_tweezer_1064_rampdown2_end,
                                paint=True,keep_trap_frequency_constant=True)
                
                # tweezer evap 3 with constant trap frequency
                self.ttl.pd_scope_trig.pulse(1.e-6)
                self.tweezer.ramp(t=self.p.t_tweezer_1064_rampdown3,
                                v_start=tweezer_vpd1_to_vpd2(self.p.v_pd_tweezer_1064_rampdown2_end),
                                v_end=self.p.v_pd_tweezer_1064_rampdown3_end,
                                paint=True,keep_trap_frequency_constant=True,low_power=True)
                
                # self.outer_coil.ramp(t=self.p.t_feshbach_field_ramp2,
                #                      i_start=self.p.i_evap3_current,
                #                      i_end=self.p.i_end)
                
                self.lightsheet.off()
                self.tweezer.off()

                delay(self.p.t_tof)
                self.abs_image()

                self.outer_coil.off()
                self.outer_coil.discharge()

            @kernel
            def run(self):
                self.init_kernel()
                self.load_2D_mot(self.p.t_2D_mot_load_delay)
                self.scan()
                self.mot_observe()

            def analyze(self):
                import os
                expt_filepath = os.path.abspath(__file__)
                self.end(expt_filepath)
        """)
        return script

In [2]:
eBuilder = ExptBuilder()

In [3]:
f_cat_eye = np.linspace(70.0,70.7,12)*1.e6
for i in f_cat_eye:
    f_list = [i,80.e6]
    print(f_list)
    eBuilder.write_experiment_to_file(eBuilder.fringe_scan_expt(f_list = f_list))
    eBuilder.run_expt()

[70000000.0, 80000000.0]
0  500 values of t_tof. 500 total shots. 1500 total images expected.
andor
Acknowledged camera ready signal.
Camera is ready.
 Run ID: 11683
Parameters saved, data closed.
Done!
 
[70063636.36363636, 80000000.0]
0  500 values of t_tof. 500 total shots. 1500 total images expected.
andor
Acknowledged camera ready signal.
Camera is ready.
 Run ID: 11684
Parameters saved, data closed.
Done!
 
[70127272.72727272, 80000000.0]
0  500 values of t_tof. 500 total shots. 1500 total images expected.
andor
Acknowledged camera ready signal.
Camera is ready.
 Run ID: 11685
Parameters saved, data closed.
Done!
 
[70190909.0909091, 80000000.0]
0  500 values of t_tof. 500 total shots. 1500 total images expected.
andor
Acknowledged camera ready signal.
Camera is ready.
 Run ID: 11686
Parameters saved, data closed.
Done!
 
[70254545.45454545, 80000000.0]
0  500 values of t_tof. 500 total shots. 1500 total images expected.
andor
Acknowledged camera ready signal.
Camera is ready.
 R